In [ ]:
#hide
#skip
%config Completer.use_jedi = False
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [ ]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [ ]:
# default_exp callback.core

In [ ]:
# export
# Python native modules
import os
from typing import *
# Third party libs
from fastcore.all import *
from fastai.callback.core import *
from fastai.data.all import *
# Local modules

In [ ]:
from fastai.learner import *
from fastai.test_utils import *

# Callback Core
> Exntensions of the fastai learner training loop API for Agents and Sources

The fastai learner has a training loop that looks like...

In [ ]:
synth_learner().show_training_loop()

Start Fit
   - before_fit     : [TrainEvalCallback, Recorder]
  Start Epoch Loop
     - before_epoch   : [Recorder]
    Start Train
       - before_train   : [TrainEvalCallback, Recorder]
      Start Batch Loop
         - before_batch   : []
         - after_pred     : []
         - after_loss     : []
         - before_backward: []
         - before_step    : []
         - after_step     : []
         - after_cancel_batch: []
         - after_batch    : [TrainEvalCallback, Recorder]
      End Batch Loop
    End Train
     - after_cancel_train: [Recorder]
     - after_train    : [Recorder]
    Start Valid
       - before_validate: [TrainEvalCallback, Recorder]
      Start Batch Loop
         - **CBs same as train batch**: []
      End Batch Loop
    End Valid
     - after_cancel_validate: [Recorder]
     - after_validate : [Recorder]
  End Epoch Loop
   - after_cancel_epoch: []
   - after_epoch    : [Recorder]
End Fit
 - after_cancel_fit: []
 - after_fit      : []


It would be great to have this as a generic object that other parts of fastrl can leverage.

In [ ]:
# export
defaults.loop_callbacks=L()

_loop = ['Start Fit', 'before_fit', 'Start Epoch Loop', 'before_epoch', 'Start Train', 'before_train',
         'Start Batch Loop', 'before_batch', 'after_pred', 'after_loss', 'before_backward', 'before_step',
         'after_step', 'after_cancel_batch', 'after_batch','End Batch Loop','End Train',
         'after_cancel_train', 'after_train', 'Start Valid', 'before_validate','Start Batch Loop',
         '**CBs same as train batch**', 'End Batch Loop', 'End Valid', 'after_cancel_validate',
         'after_validate', 'End Epoch Loop', 'after_cancel_epoch', 'after_epoch', 'End Fit',
         'after_cancel_fit', 'after_fit']

class Loop(GetAttr):
    _loop,_default,_events=_loop,'_base',None
    def __init__(self,cbs=None,persistent=False,**kwargs):
        store_attr(but='cbs')
        self.cbs=L()
        self.add_cbs(L(defaults.loop_callbacks)+L(cbs))
        self("after_create")
        
    def _grab_cbs(self, cb_cls): return L(cb for cb in self.cbs if isinstance(cb, cb_cls))

    def add_cbs(self, cbs):
        L(cbs).map(self.add_cb)
        return self

    def remove_cbs(self, cbs):
        if not self.persistent: L(cbs).map(self.remove_cb)
        return self

    def add_cb(self, cb):
        if isinstance(cb, type): cb = cb()
        setattr(cb,self._default,self)
        setattr(self, cb.name, cb)
        self.cbs.append(cb)
        return self

    def remove_cb(self, cb):
        if isinstance(cb, type): self.remove_cbs(self._grab_cbs(cb))
        else:
            setattr(cb,self._default,None)
            if hasattr(self, cb.name): delattr(self, cb.name)
            if cb in self.cbs: self.cbs.remove(cb)
        return self

    def _with_events(self, f, event_type, ex, final=noop):
        try: self(f'before_{event_type}');  f()
        except ex: self(f'after_cancel_{event_type}')
        self(f'after_{event_type}');  final()
    
    @contextmanager
    def added_cbs(self, cbs):
        self.add_cbs(cbs)
        try: yield
        finally: self.remove_cbs(cbs)

    @contextmanager
    def removed_cbs(self, cbs):
        self.remove_cbs(cbs)
        try: yield self
        finally: self.add_cbs(cbs)

    def ordered_cbs(self, event): return [cb for cb in self.cbs.sorted('order') if hasattr(cb, event)]
    def __call__(self, event_name): L(event_name).map(self._call_one)

    def _call_one(self, event_name):
        if not hasattr(self._events, event_name): raise Exception(f'missing {event_name}')
        for cb in self.cbs.sorted('order'): cb(event_name)
            

    def show_loop(self):
        indent = 0
        for s in self._loop:
            s=s.replace('event.','')
            if s.startswith('Start'): print(f'{" "*indent}{s}'); indent += 2
            elif s.startswith('End'): indent -= 2; print(f'{" "*indent}{s}')
            else: print(f'{" "*indent} - {s:15}:', self.ordered_cbs(s))

Ok so lets try about making a custom callback loop!

In [ ]:
_events = L.split('create do_this then_this and_finally_this')
_events = _events.map(lambda s:'cancel_'+s)+ _events.map(lambda s:'before_'+s) + _events.map(lambda s:'after_'+s)
mk_class('test_event', **_events.map_dict(),
         doc="All possible events as attributes to get tab-completion and typo-proofing")


class TstLooper(Loop):
    _events=test_event
    _loop=L(['Start doing this','before_do_this','after_do_this','End doing this',
             'Start then doing this','before_then_this','after_then_this',
             'Start finally this','before_and_finally_this','after_and_finally_this',
             'End finally this','End doing this'])
    def do_this(self):          return
    def then_this(self):        return
    def and_finally_this(self): return 
    
    def fit(self):
        self._with_events(self.do_this,'do_this',Exception)
        self._with_events(self.then_this,'then_this',Exception)
        self._with_events(self.and_finally_this,'and_finally_this',Exception)

In [ ]:
TstLooper().show_loop()

Start doing this
   - before_do_this : []
   - after_do_this  : []
End doing this
Start then doing this
   - before_then_this: []
   - after_then_this: []
  Start finally this
     - before_and_finally_this: []
     - after_and_finally_this: []
  End finally this
End doing this


Ok so we have a custom callback loop, now we want to have some callbacks that get executed!

In [ ]:
# export
_events = L.split('')

mk_class('loop_event', **_events.map_dict(),
         doc="All possible events as attributes to get tab-completion and typo-proofing")

_all_ = ['loop_event']

In [ ]:
#export
_inner_loop = "".split()

Object `Stateful` not found.


In [ ]:
#export
@funcs_kwargs(as_method=True)
class LoopCallback(Stateful,GetAttr):
    "Basic class handling tweaks of a callback loop by changing a `obj` in various events"
    order,_default,obj,run,run_train,run_valid = 0,'obj',None,True,True,True
    end_event='after_fit'
    _methods = _events

    def __init__(self, **kwargs): assert not kwargs, f'Passed unknown events: {kwargs}'
    def __repr__(self): return type(self).__name__

    def __call__(self, event_name,**kwargs):
        "Call `self.{event_name}` if it's defined"
        _run = (event_name not in _inner_loop or (self.run_train and getattr(self, 'training', True)) or
               (self.run_valid and not getattr(self, 'training', False)))
        res=None
        if self.run and _run: 
            res=getattr(self, event_name, noop)(**kwargs)
        if event_name==self.end_event: self.run=True #Reset self.run to True at each end of fit
        return res

    def __setattr__(self, name, value):
        if hasattr(self.obj,name):
            warn(f"You are shadowing an attribute ({name}) that exists in the {self._default}. Use `self.{self._default}.{name}` to avoid this")
        super().__setattr__(name, value)

    @property
    def name(self):
        "Name of the `LoopCallback`, camel-cased and with '*LoopCallback*' removed"
        return class2attr(self, 'LoopCallback')

For a custom callback looper, we need 2 things: `_events` and `_inner_loop`.
If an event is in the `_inner_loop` then we want to skip executing it since it is going to be 
manually executed by some other event.

In [ ]:
_events = L.split('create do_this then_this and_finally_this')
_events = _events.map(lambda s:'cancel_'+s)+ _events.map(lambda s:'before_'+s) + _events.map(lambda s:'after_'+s)
mk_class('event', **_events.map_dict(),
         doc="All possible events as attributes to get tab-completion and typo-proofing")

class CustomCallback(LoopCallback):
    _default,some_obj='some_class',None
    _methods=_events

#     def do_this(self):          return print('do_this')
#     def then_this(self):        return print('then_this')
#     def and_finally_this(self): return print('and_finally_this')
    def before_do_this(self):          return print('before_do_this')
    def before_then_this(self):        return print('before_then_this')
    def before_and_finally_this(self): return print('before_and_finally_this')
    def after_do_this(self):           return print('after_do_this')
    def after_then_this(self):         return print('after_then_this')
    def after_and_finally_this(self):  return print('after_and_finally_this')

In [ ]:
CustomCallback()._methods

(#12) ['cancel_create','cancel_do_this','cancel_then_this','cancel_and_finally_this','before_create','before_do_this','before_then_this','before_and_finally_this','after_create','after_do_this'...]

In [ ]:
TstLooper().do_this()

In [ ]:
looper=TstLooper(cbs=CustomCallback())

In [ ]:
looper.show_loop()

Start doing this
   - before_do_this : [CustomCallback]
   - after_do_this  : [CustomCallback]
End doing this
Start then doing this
   - before_then_this: [CustomCallback]
   - after_then_this: [CustomCallback]
  Start finally this
     - before_and_finally_this: [CustomCallback]
     - after_and_finally_this: [CustomCallback]
  End finally this
End doing this


In [ ]:
looper.fit()

before_do_this
after_do_this
before_then_this
after_then_this
before_and_finally_this
after_and_finally_this


In [5]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbdev.cli import make_readme
    make_readme()
    notebook2script()
    notebook2html()

converting /home/fastrl_user/fastrl/nbs/index.ipynb to README.md
Converted 00_core.ipynb.
Converted 00_nbdev_extension.ipynb.
Converted 03_callback.core.ipynb.
Converted 04_agent.ipynb.
Converted 05_data.test_async.ipynb.
Converted 05a_data.block.ipynb.
Converted 05b_data.gym.ipynb.
Converted 06a_memory.experience_replay.ipynb.
Converted 06f_memory.tensorboard.ipynb.
Converted 10a_agents.dqn.core.ipynb.
Converted 10b_agents.dqn.targets.ipynb.
Converted 10c_agents.dqn.double.ipynb.
Converted 10d_agents.dqn.dueling.ipynb.
Converted 10e_agents.dqn.categorical.ipynb.
Converted 11a_agents.policy_gradient.ppo.ipynb.
Converted 20_test_utils.ipynb.
Converted index.ipynb.
Converted nbdev_template.ipynb.
converting: /home/fastrl_user/fastrl/nbs/03_callback.core.ipynb
